### Info
based on notebook3\
checked the input quantizer (14,6)

### Load packages and the model

In [1]:
import tensorflow as tf
import numpy
import h5py
from qkeras.utils import _add_supported_quantized_objects
from qkeras import quantized_bits
co = {}; _add_supported_quantized_objects(co)

model_path = f"/eos/project/c/cms-l1ml/public/Pawel/axol1tl_model.h5"

model = tf.keras.models.load_model(model_path, custom_objects=co)
#model_config = model.get_config()

data_path = f"/eos/project/c/cms-l1ml/public/Pawel/complete.h5"
data_file = h5py.File(data_path, 'r')

2025-09-24 15:22:59.438117: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2025-09-24 15:23:01.145380: E tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:9342] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2025-09-24 15:23:01.145437: E tensorflow/compiler/xla/stream_executor/cuda/cuda_fft.cc:609] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2025-09-24 15:23:01.155302: E tensorflow/compiler/xla/stream_executor/cuda/cuda_blas.cc:1518] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2025-09-24 15:23:02.226889: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2025-09-24 15:23:02.237623: I tensorflow/core/platform/cpu_feature_guard.cc:182] This Tens

### Copy weights

In [2]:
ds_weights = data_file['model']['trimmed_encoder']['model_weights']
for layer in iter(model.layers):
    #print(layer.name)
    if layer.name in ('input_1','subtract','dot'): continue

    if layer.name == 'q_dense_4':
        weights_group = ds_weights['q_dense_4']['variational_auto_encoder']['vae__encoder']['q_dense_4']
        layer.set_weights((weights_group['kernel:0'], weights_group['bias:0']))
        continue

    weights_group = ds_weights[layer.name][layer.name]
    layer.set_weights((weights_group['kernel:0'], weights_group['bias:0']))

### Look at the data

In [3]:
data_file['data']["Background_data"]["Test"]["DATA"].shape

(4511092, 19, 3)

In [4]:
X = data_file['data']["Background_data"]["Test"]["DATA"][:2048]
scale_data = data_file['data']['Normalisation']['norm_scale'][:].flatten()
offset_data =  data_file['data']['Normalisation']['norm_bias'][:].flatten()

From qkeras github "quantizers.py":\
the behavior of quantized_bits is different than Catapult HLS ac_fixed
or Vivado HLS ap_fixed. For ac_fixed<word_length, integer_lenth, signed>,
when signed = true, it is equavlent to
quantized_bits(word_length, integer_length-1, keep_negative=True)


In [13]:
#Quantizers
input_quantizer = quantized_bits(14, 6, alpha=1)
output_quantizer = quantized_bits(18,14,alpha=1)

In [17]:
for i in range(X.shape[0]):
    for j in range(X.shape[1]):
        row = X[i,j]
        diff = row - input_quantizer(row).numpy()
        if numpy.any(diff):
            print(i,j,'\t',diff)

0 6 	 [ 0.          0.         -0.00390625]
1 6 	 [ 0.          0.         -0.00390625]
2 6 	 [ 0.          0.         -0.00390625]
3 5 	 [0.         0.         0.00390625]
3 6 	 [ 0.          0.         -0.00390625]
4 6 	 [ 0.          0.         -0.00390625]
5 6 	 [ 0.          0.         -0.00390625]
6 6 	 [ 0.          0.         -0.00390625]
7 5 	 [0.         0.         0.00390625]
8 6 	 [ 0.          0.         -0.00390625]
9 6 	 [ 0.          0.         -0.00390625]
10 5 	 [ 0.          0.         -0.00390625]
10 6 	 [ 0.          0.         -0.00390625]
11 6 	 [ 0.          0.         -0.00390625]
12 6 	 [ 0.          0.         -0.00390625]
14 6 	 [ 0.          0.         -0.00390625]
15 5 	 [0.         0.         0.00390625]
15 6 	 [ 0.          0.         -0.00390625]
16 6 	 [ 0.          0.         -0.00390625]
17 6 	 [ 0.          0.         -0.00390625]
18 6 	 [ 0.          0.         -0.00390625]
19 6 	 [ 0.          0.         -0.00390625]
20 6 	 [ 0.          0.       

In [ ]:
i = 8
(X[:,i]*scale_data.reshape(-1,3)[i] + offset_data.reshape(-1,3)[i])

array([[  0.,   0.,   0.],
       [  0.,   0.,   0.],
       [  0.,   0.,   0.],
       ...,
       [  0.,   0.,   0.],
       [ 21., 207., 308.],
       [  0.,   0.,   0.]])